In [ ]:

# yeni hali 

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import os

import tensorflow as tf

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  LSTM , BatchNormalization

from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, RepeatVector, Concatenate, Attention

from tensorflow.keras.models import Model

from sklearn.preprocessing import MinMaxScaler


In [ ]:
import yfinance as yf

In [ ]:
import os

if os.path.exists('datas'):
    #sil
    os.system('rm -rf datas')

In [ ]:
period = '3000d'


features = ['Open', 'High', 'Low', 'Close','Adj Close', 'Volume']
#features = [ 'Close',]  # Sadece kapanış fiyatını kullanın

#tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'TSLA', 'NVDA', 'PYPL', 'ADBE','BTC-USD', 'ETH-USD', 'XRP-USD', 'LTC-USD','BCH-USD', 'BNB-USD', 'LINK-USD', 'ADA-USD', 'XLM-USD', 'SOL-USD', 'TRX-USD']

tickers = ['SOL-USD']




In [ ]:
def indir_ve_df_olustur(tickers, period):
    # Her bir hisse senedi için boş bir sözlük oluşturun
    pariteler = {}

    # Her bir hisse senedi için döngü oluşturun ve verileri indirin
    for ticker in tickers:
        try:
            # Hisse senedi verilerini indirin
            veri = yf.download(ticker, period=period)

            veri = veri[features]   
            # Veriyi sözlüğe ekleyin
            pariteler[ticker] = veri
            
            
        except Exception as e:
            print(f"{ticker} için veri indirilirken bir hata oluştu: {str(e)}")

    # Her bir hisse senedi için ayrı bir veri çerçevesi oluşturun
    df_listesi = [veri for veri in pariteler.values()]

    return df_listesi

In [ ]:
#pariteler adında klasör oluştur :

import os 

if not os.path.exists('datas/pariteler'):
    os.makedirs('datas/pariteler')



In [ ]:
datasets = indir_ve_df_olustur(tickers, period)

#pariteler içine kaydet 

for i in range(len(datasets)):
    datasets[i].to_csv(f'datas/pariteler/{tickers[i]}.csv')
    print(f'{tickers[i]} verisi csv olarak kaydedildi')


In [ ]:
scaler  = MinMaxScaler()

#her bir veri çerçevesi için scaled dataset oluştur

if not os.path.exists('datas/scaled_datasets'):
    os.makedirs('datas/scaled_datasets')

scaled_datasets = []

for i in range(len(datasets)):
    scaled_datasets.append(scaler.fit_transform(datasets[i]))
    np.save(f'datas/scaled_datasets/{tickers[i]}.npy', scaled_datasets[i])
    print(f'{tickers[i]} verisi scaled edilerek kaydedildi')
    

In [ ]:
# her bir veri çerçevesi için veriyi dizilere bölme

seq_length = 60  # Son 60 günü kullanarak
pred_length = 15  # Önümüzdeki 15 günü tahmin etme

def create_sequences(data, seq_length, pred_length):
    X, y = [], []
    for i in range(len(data)-seq_length-pred_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length:i+seq_length+pred_length])
    return np.array(X), np.array(y)

# her bir veri çerçevesi için veriyi dizilere bölme

if not os.path.exists('datas/sequences'):
    os.makedirs('datas/sequences')

X, y = [], []

for i in range(len(scaled_datasets)):
    X_, y_ = create_sequences(scaled_datasets[i], seq_length, pred_length)
    X.append(X_)
    y.append(y_)
    np.save(f'datas/sequences/X_{tickers[i]}.npy', X_)
    np.save(f'datas/sequences/y_{tickers[i]}.npy', y_)
    print(f'{tickers[i]} verisi dizilere bölünerek kaydedildi')
    


In [ ]:
# her bir veri çerçevesi için veriyi train ve test olarak bölme

X_train, X_test, y_train, y_test = [], [], [], []

for i in range(len(scaled_datasets)):
    X_train_, X_test_, y_train_, y_test_ = X[i][:-pred_length], X[i][-pred_length:], y[i][:-pred_length], y[i][-pred_length:]
    X_train.append(X_train_)
    X_test.append(X_test_)
    y_train.append(y_train_)
    y_test.append(y_test_)
    

In [ ]:
# her bir veri çerçevesi için  seq2seq attention modeli oluştur

# Encoder
encoder_inputs = Input(shape=(seq_length, len(features)))
encoder_lstm = LSTM(100, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]


# Decoder
decoder_inputs = RepeatVector(pred_length)(encoder_outputs)
decoder_lstm = LSTM(100, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)


# Dikkat Mekanizması
attention_layer = Attention(use_scale=True)
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_outputs])


# TimeDistributed Dense
decoder_dense = TimeDistributed(Dense(len(features)))
decoder_outputs = decoder_dense(decoder_concat_input)

def create_model(encoder_inputs, decoder_outputs):
    model = Model(encoder_inputs, decoder_outputs)
    return model
    


In [ ]:
# her bir veri çerçevesi için modeli oluştur ve eğit

if not os.path.exists('datas/models'):
    os.makedirs('datas/models')

models = []
batch_size = 64
epochs = 100

for i in range(len(scaled_datasets)):
    # modeli oluştur
    model = create_model(encoder_inputs, decoder_outputs)
    
    # modeli derle
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # modeli listeye ekle
    models.append(model)
    
    # modeli eğit
    models[i].fit(X_train[i], y_train[i], batch_size=batch_size, epochs=epochs, validation_data=(X_test[i], y_test[i]))
    
    # modeli kaydet
    models[i].save(f'datas/models/{tickers[i]}_model.h5')
    print(f'{tickers[i]} modeli eğitildi ve kaydedildi')


In [ ]:
# modelerden bir tanesini png kaydedelim
from tensorflow.keras.utils import plot_model
plot_model(models[0], to_file='datas/model.png', show_shapes=True, show_layer_names=True , rankdir='TB', expand_nested=True, dpi=96, )

In [ ]:
# Son veri noktasını kullanarak geleceği tahmin etme

if not os.path.exists('datas/predictions'):
    os.makedirs('datas/predictions')
    
    
for i in range(len(scaled_datasets)):
    # modeli yükle
    model = tf.keras.models.load_model(f'datas/models/{tickers[i]}_model.h5')
    
    # geleceği tahmin et
    predicted = model.predict(X_test[i])
    
    # tahminleri kaydet
    np.save(f'datas/predictions/{tickers[i]}_predictions.npy', predicted)
    print(f'{tickers[i]} için tahminler kaydedildi')
    


    

In [ ]:
# tahminleri gerçek verilerle karşılaştır

for i in range(len(scaled_datasets)):
    # gerçek verileri yükle
    real = y_test[i]
    
    # tahminleri yükle
    predicted = np.load(f'datas/predictions/{tickers[i]}_predictions.npy')
    
    # gerçek ve tahmin edilen verileri ters ölçeklendir
    real = scaler.inverse_transform(real.reshape(-1, len(features)))
    predicted = scaler.inverse_transform(predicted.reshape(-1, len(features)))
    
    # gerçek ve tahmin edilen verileri veri çerçevesine dönüştür
    real_df = pd.DataFrame(real, columns=features)
    predicted_df = pd.DataFrame(predicted, columns=features)
    
    # gerçek ve tahmin edilen tüm verileri çizdir
    plt.figure(figsize=(14, 7))
    for feature in features:
        plt.plot(real_df[feature], label=f'Real {feature}')
        plt.plot(predicted_df[feature], label=f'Predicted {feature}')
    plt.title(f'{tickers[i]} Real vs Predicted')
    
    plt.legend()
    
    plt.savefig(f'datas/charts/{tickers[i]}_real_vs_predicted.png')
    